In [1]:
!which python

/Users/apabook/Desktop/ds-deployment/.venv/bin/python


In [2]:
import pandas as pd
import requests
import json
import pprint


/Users/apabook/Desktop/ds-deployment/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Testing HL Api for PUSH transfer

---

## Consts

In [3]:
data_space_provider = "http://127.0.0.1:1234"
data_space_consumer = "http://127.0.0.1:1235"
context_broker = "http://fiware-orion:1026"
streaming_processor = "http://streaming-testing-service:1237"

catalog_id = ""
dataservice_id = ""
agreement_id = ""
agreement_pid = ""

## Setup Catalog, Dataservice and agreements

In [4]:
payload = {
    "foaf:homepage": "My catalog in Dataspace provider",
    "dct:title": "My catalog in Dataspace provider"
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/catalogs"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
catalog_id = response_as_json["@id"]

pprint.pprint(response.json())

{'@context': 'https://w3id.org/dspace/2024/1/context.json',
 '@id': 'aeec1757-a93c-4d09-8057-9008240d9b36',
 '@type': 'dcat:Catalog',
 'dcat:dataset': [],
 'dcat:keyword': '',
 'dcat:service': [],
 'dcat:theme': '',
 'dct:conformsTo': None,
 'dct:creator': None,
 'dct:description': [],
 'dct:identifier': 'aeec1757-a93c-4d09-8057-9008240d9b36',
 'dct:issued': '2024-12-02T14:26:17.607127',
 'dct:modified': None,
 'dct:title': 'My catalog in Dataspace provider',
 'dspace:extraFields': None,
 'dspace:participantId': None,
 'foaf:homepage': 'My catalog in Dataspace provider',
 'odrl:hasPolicy': None}


In [5]:
suscription_payload = {
    "description": "Air Quality Unit Subscription",
    "subject": {
        "entities":[
            {"idPattern":".*","type":"AirQualityUnit"}],
        "condition": {
            "attrs":["CO"]
        }
    },
    "notification": {
        "http": {
            "url":"$data_url",
        },
    },
    "expires":"2040-01-01T14:00:00.00Z",
    "throttling":5
}
payload = {
    "dcat:endpointURL": context_broker + "/v2/subscriptions",
    "dcat:endpointDescription": json.dumps(suscription_payload),
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/catalogs/" + catalog_id + "/data-services"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
dataservice_id = response_as_json["@id"]

pprint.pprint(response.json())

{'@context': 'https://w3id.org/dspace/2024/1/context.json',
 '@id': 'fc071617-9ac7-41ae-81da-7fe8865f7c36',
 '@type': 'dcat:DataService',
 'dcat:endpointDescription': '{"description": "Air Quality Unit Subscription", '
                             '"subject": {"entities": [{"idPattern": ".*", '
                             '"type": "AirQualityUnit"}], "condition": '
                             '{"attrs": ["CO"]}}, "notification": {"http": '
                             '{"url": "$data_url"}}, "expires": '
                             '"2040-01-01T14:00:00.00Z", "throttling": 5}',
 'dcat:endpointURL': 'http://fiware-orion:1026/v2/subscriptions',
 'dcat:keyword': '',
 'dcat:theme': '',
 'dct:conformsTo': None,
 'dct:creator': None,
 'dct:description': [],
 'dct:identifier': 'fc071617-9ac7-41ae-81da-7fe8865f7c36',
 'dct:issued': '2024-12-02T14:26:18.266731',
 'dct:modified': None,
 'dct:title': None,
 'dspace:extraFields': None,
 'odrl:hasPolicy': None}


In [6]:
payload = {
    "dataServiceId": dataservice_id
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/agreements"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
agreement_id = response_as_json["agreement_id"]
agreement_pid = "urn:uuid:" + agreement_id

pprint.pprint(response.json())

{'agreement_id': '92a196cc-7f3a-4c0d-836c-2dba5c9ccb17',
 'data_service_id': 'fc071617-9ac7-41ae-81da-7fe8865f7c36',
 'identity': None,
 'identity_token': None}


## Setup transfer

In [7]:
url = data_space_consumer + "/api/v1/setup-transfer"

payload = ""
headers = {}
response = requests.request("POST", url, headers=headers, data=payload)
callbackAddress = response.json()["callbackAddress"]
callbackId = response.json()["callbackId"]
consumerPid = response.json()["consumerPid"]

pprint.pprint(response.json())

{'callbackAddress': 'http://ds-consumer:1235/754abdb3-cded-4499-9ccb-35f1c9949079',
 'callbackId': '754abdb3-cded-4499-9ccb-35f1c9949079',
 'consumerPid': 'urn:uuid:de019499-0421-4b5d-bea8-5cfa00aede8c'}


## Request transfer

In [8]:
url = data_space_consumer + "/api/v1/request-transfer"

payload = json.dumps({
    "agreementId": agreement_pid,
    "format": "fiware+push",
    "dataAddress": {
        "@type": "dspace:DataAddress",
        "dspace:endpoint": streaming_processor + "/data-client",
        "dspace:endpointType": "HTTP",
        "dspace:endpointProperties": []
    },
    "callbackAddress": callbackAddress,
    "callbackId": callbackId,
    "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())


{'consumerPid': 'urn:uuid:de019499-0421-4b5d-bea8-5cfa00aede8c',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:de019499-0421-4b5d-bea8-5cfa00aede8c',
                     'dspace:providerPid': 'urn:uuid:3668821f-aa3c-4a9f-8662-386372f57b67',
                     'dspace:state': 'dspace:REQUESTED'}}


#### For seeing how the push service is working, please check the logs in the streaming-testing-service

## Suspend transfer

In [9]:
url = data_space_consumer + "/api/v1/suspend-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:de019499-0421-4b5d-bea8-5cfa00aede8c',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:de019499-0421-4b5d-bea8-5cfa00aede8c',
                     'dspace:providerPid': 'urn:uuid:3668821f-aa3c-4a9f-8662-386372f57b67',
                     'dspace:state': 'dspace:SUSPENDED'}}


## Test data access has been suspended


#### For seeing how the push service is no longer receiving data, please check the logs in the streaming-testing-service

## Restart transfer

In [33]:
url = data_space_consumer + "/api/v1/restart-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:5d2ec721-4cb2-4735-9141-59acfbfded12',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:5d2ec721-4cb2-4735-9141-59acfbfded12',
                     'dspace:providerPid': 'urn:uuid:3712845c-27c1-4532-916d-70ee1098ee67',
                     'dspace:state': 'dspace:STARTED'}}


## Test data is accessible again

#### For seeing how the push service is working, please check the logs in the streaming-testing-service

## Complete transfer

In [34]:
url = data_space_consumer + "/api/v1/complete-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:5d2ec721-4cb2-4735-9141-59acfbfded12',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:5d2ec721-4cb2-4735-9141-59acfbfded12',
                     'dspace:providerPid': 'urn:uuid:3712845c-27c1-4532-916d-70ee1098ee67',
                     'dspace:state': 'dspace:COMPLETED'}}


## Data is not accessible

#### For seeing how the push service is no longer receiving data, please check the logs in the streaming-testing-service

## Protocol state cannot change

In [35]:
url = data_space_consumer + "/api/v1/restart-transfer"

payload = json.dumps({
  "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())

{'consumerPid': 'urn:uuid:5d2ec721-4cb2-4735-9141-59acfbfded12',
 'error': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
           '@type': 'dspace:TransferError',
           'dspace:code': 'TRANSFER_ERROR_CODE',
           'dspace:consumerPid': '123',
           'dspace:providerPid': '123',
           'dspace:reason': ['Protocol Error. dspace:TransferRequestMessage is '
                             'not allowed here. Current state is '
                             'dspace:COMPLETED']}}
